In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from alphagogoat.delphox import Delphox
# from alphagogoat.utils import DataExtractor
# from alphagogoat.embedder import process_battle
from pathlib import Path
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import json
import numpy as np
import json
import logging
import copy
from enum import IntEnum
from collections import Counter

from poke_env.environment.battle import Battle
from poke_env.environment.move import Move
import re
from poke_env.environment.pokemon import Pokemon
from poke_env.environment.effect import Effect
import torch


# from alphagogoat.pokedex import make_pokedex
LOGGER = logging.getLogger('poke-env')
GEN = 8
# POKEDEX = make_pokedex()

In [3]:
def process_battle(battle_json: str) -> list[Battle]:
    with open(battle_json) as f:
        battle_data = json.load(f)
    history = battle_data['log'].split('\n')
    curr_battle = Battle(battle_data['id'], battle_data['p1'], LOGGER, GEN)
    curr_battle._opponent_username = battle_data['p2']
    battle_objects = []
    for line in history:
        try:
            curr_battle._parse_message(line.split('|'))
            if line.split('|')[1] == 'turn':
                battle_objects.append(copy.deepcopy(curr_battle))
        except:
            continue

    return battle_objects


In [4]:
battle_paths = [f"cache/replays/{file.name}" for file in Path("cache/replays").iterdir() if file.name.endswith(".json")]

def get_turns(battle_json_path):
    return DataExtractor(battle_json_path)

# DEBUGGING
battle_paths = battle_paths[:5]
extractors = Parallel(n_jobs=4)(delayed(get_turns)(battle_json_path) for battle_json_path in tqdm(battle_paths))

  0%|          | 0/5 [00:00<?, ?it/s]

NameError: name 'DataExtractor' is not defined

In [ ]:
pokedex = {}
for filepath in tqdm(list(Path("cache/teams").iterdir())):
    with open(filepath, 'r') as f:
        data = json.load(f)
    for key, value in data.items():
        if key not in pokedex:
            pokedex[key] = {k: [] for k in value.keys()}
        for k, v in value.items():
            pokedex[key][k].append(v)

for pokemon, dicts in tqdm(pokedex.items()):
    pokedex[pokemon]['level'] = int(np.round(np.mean(pokedex[pokemon]['level'])))
    for k1 in ['abilities', 'moves', 'items']:
        d = {}
        if k1 in dicts:
            for dict_ in dicts[k1]:
                if not isinstance(dict_, dict):
                    continue
                for k2, prob in dict_.items():
                    if k2 not in d:
                        d[k2] = []
                    d[k2].append(prob)
            for k2, probs in d.items():
                d[k2] = np.mean(probs)
        pokedex[pokemon][k1] = d

In [ ]:
pokedex['Pikachu']

In [ ]:
for pokemon, data in pokedex.items():
    if 'ivs' in data:
        print(pokemon)
        print(data['ivs'])
        break

In [ ]:
item_catalog = set()
for pokemon, data in pokedex.items():
    item_catalog.update(data['items'])

item_catalog = sorted(list(item_catalog))
Item = IntEnum('Item', item_catalog)

In [ ]:
ability_catalog = set()
for pokemon, data in pokedex.items():
    ability_catalog.update(data['abilities'])

ability_catalog = sorted(list(ability_catalog))
Ability = IntEnum('Ability', ability_catalog)
list(Ability)

In [ ]:
move_catalog = set()
move_enum = set()
for pokemon, data in pokedex.items():
    move_catalog.update(data['moves'])
    for move in data['moves']:
        move_enum.add(re.sub("\s|-|'", "", move.lower()))


MoveEnum = IntEnum("MoveEnum", list(sorted(move_enum)))

move_catalog = [Move(re.sub("\s|-|'", "", move.lower()), 8) for move in sorted(list(move_catalog))]
volatile_status_catalog = set()
for move in move_catalog:
    if move.volatile_status is None:
        continue
    volatile_status_catalog.add(move.volatile_status)

VolatileStatus = IntEnum("VolatileStatus", list(sorted(volatile_status_catalog)))

In [ ]:
list(MoveEnum)

In [ ]:
max_slots = 0
max_status = 0
max_on_hit = 0
max_boosts = 0
max_chance = 0
max_volatile_status = 0
max_self = 0
# NOTE: ignore dustproof
keys = set()
for move in move_catalog:
    if move.secondary:
        if any('self' in d for d in move.secondary):
            print(move)
            print(move.secondary)
        if any('boosts' in d for d in move.secondary):
            print(move)
            print(move.secondary)
        for d in move.secondary:
            keys.update(d.keys())
        # keys = Counter()
        # for d in move.secondary:
        #     for k in d:
        #         if k == 'chance':
        #             continue
        #         keys[k] += 1
        # print(keys.most_common(1))
        # num_slots = 0
        # for d in move.secondary:
        #     num_slots += len(d)
        #     keys.update(d.keys())
        #     # print(list(d.keys())[0])
        #     if 'status' in d:
        #         print(d)
        # max_slots = max(num_slots, max_slots)
        # print(f"{}")
# print(f"{max_slots=}")
print(f"{keys=}")

In [ ]:
list(VolatileStatus)

In [ ]:
len(item_catalog)

In [ ]:
max_abilities = 0
max_items = 0
max_moves = 0
for pokemon, data in pokedex.items():
    max_abilities = max(len(data['abilities']), max_abilities)
    max_moves = max(len(data['moves']), max_moves)
    max_items = max(len(data['items']), max_items)
print(f"{max_abilities=}\n{max_items=}\n{max_moves=}")

In [ ]:
dynamax_moves = [move.dynamaxed]

In [ ]:
len(move_catalog)

In [ ]:
turns = extractors[0].turns
extractor = extractors[0]
embedding = extractor.embed(turns[3])
embedding.shape

In [ ]:
battle_json = "cache/replays/gen8randombattle-1123651831.json"
turns = process_battle(battle_json)

In [ ]:
team = {}
for turn in turns:
    active = turn.active_pokemon
    team[active.species] = active
    print(active.stats)
print(team)

In [ ]:
pokedex['Incineroar'].keys()